---
layout: post
title: Drumhead
---

In [1]:
from pathlib import Path
import os
import functools
import itertools
import time

from IPython.display import HTML, Image, Video
import matplotlib.pyplot as plt
import numpy as np
from celluloid import Camera
import matplotlib.patches as patches
from scipy.integrate import odeint, solve_ivp
from simple_pid import PID
from tqdm import tqdm

ROOT = Path("./assets/img/")

if not os.path.exists(ROOT):
    os.makedirs(ROOT)

2D wave equation:

$$
\frac{\partial^2 u}{\partial t^2} = c^2(\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2})
$$

In [2]:
R = 1.
A = .1
c = 1.
center = np.array([0., 0.])
sigma = 0.2
h = 0.01

In [3]:
def plot_surface(X, Y, u, ax):
    ax.plot_surface(X, Y, u, color="royalblue")

In [22]:
def sim_2d_wave():

    X = np.arange(-R, R, h)
    Y = np.arange(-R, R, h)
    X, Y = np.meshgrid(X, Y)

    u0 = A*np.exp(-((X - center[0])**2 + (Y - center[1])**2)/sigma**2)
    inside = (X**2 + Y**2 < R**2)
    u0[~inside] = 0.

    dt = 0.7*h/(c*np.sqrt(2))

    Lambda = c*dt/h

    T = 2.

    u0 = np.pad(u0, ((1, 1), (1, 1)), 'constant', constant_values=0)
    rows, cols = u0.shape
    sol = [u0, u0]
    for _ in tqdm(enumerate(np.arange(0, T+dt, dt))):

        u_prev = sol[-1]
        u_prev_2 = sol[-2]

        u_new = np.zeros_like(u_prev)
        for i in range(1, rows-1):
            for j in range(1, cols-1):
                u_new[i, j] = 2*u_prev[i, j] - u_prev_2[i, j] + \
                Lambda**2*(u_prev[i+1,j] + u_prev[i-1,j] + u_prev[i,j+1] + u_prev[i,j-1] - 4*u_prev[i,j])

        u_new[1:-1,1:-1][~inside] = 0.
        sol.append(u_new)
    
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')

    camera = Camera(fig)
    
    for u in tqdm(sol[::2]):

        u = u[1:-1, 1:-1]
        
        plot_surface(X, Y, u, ax=ax)
        
        camera.snap()

    anim = camera.animate()
    plt.close()

    gif_path = ROOT / "2d_wave.gif"  
    anim.save(gif_path, writer="pillow", fps=10)
    return Image(url=gif_path)

In [23]:
sim_2d_wave()

406it [00:11, 36.13it/s]
100%|██████████| 204/204 [00:03<00:00, 53.99it/s]
